# ROME-Based Language Model

This notebook demonstrates how to edit a language model using the **ROME (Rank-One Model Editing)** method.

---


## Overview

Steps:
1. Install dependencies
2. Load a pretrained model (e.g., GPT-Neo-125M)
3. Apply ROME to edit a fact
4. Validate the edit with before/after comparisons

---


## 0. Install Dependencies
Run the following cell to install required packages:


In [1]:
# Uncomment and run if needed
# !pip install transformers torch rome


## 1. Load Pretrained Model
Here we load GPT-Neo-125M using Hugging Face Transformers.


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_name = "EleutherAI/gpt-neo-125M"  
print(f"Loading model: {model_name}")

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)
#model.eval()


Loading model: EleutherAI/gpt-neo-125M


GPTNeoForCausalLM(
  (transformer): GPTNeoModel(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(2048, 768)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPTNeoBlock(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPTNeoAttention(
          (attention): GPTNeoSelfAttention(
            (attn_dropout): Dropout(p=0.0, inplace=False)
            (resid_dropout): Dropout(p=0.0, inplace=False)
            (k_proj): Linear(in_features=768, out_features=768, bias=False)
            (v_proj): Linear(in_features=768, out_features=768, bias=False)
            (q_proj): Linear(in_features=768, out_features=768, bias=False)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPTNeoMLP(
          (c_fc): Linear(in_features=768, out_features=3072, bias=True)
          (c_proj): Linear(in_fe

## 2. Load CounterFact Dataset
Load the CounterFact dataset from Hugging Face.

In [ ]:
# Install datasets library if needed
# !pip install datasets

from datasets import load_dataset
import pandas as pd

# Load the CounterFact dataset
print("Loading CounterFact dataset...")
dataset = load_dataset("azhx/counterfact", split="train")
print(f"Dataset loaded: {len(dataset)} examples")

# Convert to pandas for easier manipulation
df = dataset.to_pandas()
print(f"\nDataset columns: {df.columns.tolist()}")
print(f"\nFirst example:")
print(df.iloc[0]['requested_rewrite'])


## 3. Evaluate Model Performance
Evaluate the model on key metrics: Efficacy, Paragraph, and Neighborhood scores.


In [16]:
import torch.nn.functional as F
import numpy as np

def compute_log_probs(model, tokenizer, prompt, target_tokens):
    """Compute log probabilities for target tokens given a prompt."""
    inputs = tokenizer(prompt, return_tensors="pt")
    target_ids = tokenizer(target_tokens, return_tensors="pt", add_special_tokens=False)["input_ids"][0]
    
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits[0, -1, :]  # Last token logits
        log_probs = F.log_softmax(logits, dim=-1)
        target_log_probs = log_probs[target_ids].sum().item()
    
    return target_log_probs

def efficacy_score(model, tokenizer, subject, relation, new_object):
    """
    Efficacy Score: Measures if the model correctly answers the edited fact.
    Higher score = better (model correctly predicts the new object).
    """
    prompts = [
        f"What is the {relation} of {subject}?",
        f"The {relation} of {subject} is",
        f"{subject}'s {relation} is",
    ]
    
    scores = []
    for prompt in prompts:
        # Compute log probability of the new object
        score = compute_log_probs(model, tokenizer, prompt, new_object)
        scores.append(score)
    
    return np.mean(scores)

def paragraph_score(model, tokenizer, subject, relation, new_object):
    """
    Paragraph Score: Measures coherence in longer text generation.
    Generates a paragraph and checks if it maintains consistency.
    """
    prompt = f"The {relation} of {subject} is {new_object}."
    
    inputs = tokenizer(prompt, return_tensors="pt")
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_length=100,
            num_return_sequences=1,
            temperature=0.7,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id if tokenizer.eos_token_id is not None else tokenizer.pad_token_id
        )
    
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # Check if the generated text maintains the fact
    # Simple heuristic: check if new_object appears in the generated text
    fact_maintained = new_object.lower() in generated_text.lower()
    
    # Compute perplexity-like score (lower is better for coherence)
    # We'll use a simple metric: check if the fact is mentioned
    return 1.0 if fact_maintained else 0.0

def neighborhood_score(model, tokenizer, subject, relation, new_object):
    """
    Neighborhood Score: Measures if the model maintains performance on 
    similar but unrelated facts (should not be affected by the edit).
    """
    # Define similar but unrelated facts
    neighborhood_prompts = [
        ("Germany", "capital", "Berlin"),
        ("Italy", "capital", "Rome"),
        ("Spain", "capital", "Madrid"),
    ]
    
    scores = []
    for subj, rel, obj in neighborhood_prompts:
        prompt = f"What is the {rel} of {subj}?"
        score = compute_log_probs(model, tokenizer, prompt, obj)
        scores.append(score)
    
    return np.mean(scores)

# Evaluate the model
# Use edit parameters if defined, otherwise use defaults
try:
    eval_subject = subject
    eval_relation = relation
    eval_object = new_object
    print(f"Evaluating on edit: {subject} -> {relation} -> {new_object}")
except NameError:
    # Default values for initial evaluation (before edit is defined)
    eval_subject = "France"
    eval_relation = "capital"
    eval_object = "Lyon"
    print("Evaluating on default example (France -> capital -> Lyon)")
    print("Note: Re-run this cell after defining edit parameters to evaluate your specific edit")

print("-" * 50)

# Efficacy Score
efficacy = efficacy_score(model, tokenizer, eval_subject, eval_relation, eval_object)
print(f"Efficacy Score: {efficacy:.4f}")
print("  (Higher is better - measures if edit was successful)")

# Paragraph Score
paragraph = paragraph_score(model, tokenizer, eval_subject, eval_relation, eval_object)
print(f"\nParagraph Score: {paragraph:.4f}")
print("  (1.0 = fact maintained in generation, 0.0 = not maintained)")

# Neighborhood Score
neighborhood = neighborhood_score(model, tokenizer, eval_subject, eval_relation, eval_object)
print(f"\nNeighborhood Score: {neighborhood:.4f}")
print("  (Higher is better - measures preservation of unrelated facts)")

print("-" * 50)
print(f"\nSummary:")
print(f"  Efficacy: {efficacy:.4f}")
print(f"  Paragraph: {paragraph:.4f}")
print(f"  Neighborhood: {neighborhood:.4f}")


Evaluating on default example (France -> capital -> Lyon)
Note: Re-run this cell after defining edit parameters to evaluate your specific edit
--------------------------------------------------
Efficacy Score: -25.5820
  (Higher is better - measures if edit was successful)

Paragraph Score: 1.0000
  (1.0 = fact maintained in generation, 0.0 = not maintained)

Neighborhood Score: -25.5615
  (Higher is better - measures preservation of unrelated facts)
--------------------------------------------------

Summary:
  Efficacy: -25.5820
  Paragraph: 1.0000
  Neighborhood: -25.5615


In [5]:
# Install datasets library if needed
# !pip install datasets

from datasets import load_dataset
import pandas as pd

# Load the CounterFact dataset
print("Loading CounterFact dataset...")
dataset = load_dataset("azhx/counterfact", split="train")
print(f"Dataset loaded: {len(dataset)} examples")

# Convert to pandas for easier manipulation
df = dataset.to_pandas()
print(f"\nDataset columns: {df.columns.tolist()}")
print(f"\nFirst example:")
print(df.iloc[0]['requested_rewrite'])


Loading CounterFact dataset...


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Dataset loaded: 19728 examples

Dataset columns: ['case_id', 'pararel_idx', 'requested_rewrite', 'paraphrase_prompts', 'neighborhood_prompts', 'attribute_prompts', 'generation_prompts']

First example:
{'prompt': 'The mother tongue of {} is', 'relation_id': 'P103', 'subject': 'Danielle Darrieux', 'target_new': {'id': 'Q1860', 'str': 'English'}, 'target_true': {'id': 'Q150', 'str': 'French'}}


## 4.1 Select facts to edit  
Number of edited facts can be edited by changing num_facts

In [12]:
# Select 10 facts from the dataset
num_facts = 100
selected_facts = df.head(num_facts).copy()

print(f"Selected {num_facts} facts for editing:\n")
print("=" * 80)

for idx, row in selected_facts.iterrows():
    rewrite = row['requested_rewrite']
    subject = rewrite['subject']
    prompt_template = rewrite['prompt']
    target_true = rewrite['target_true']['str']
    target_new = rewrite['target_new']['str']
    
    print(f"\nFact {idx + 1}:")
    print(f"  Subject: {subject}")
    print(f"  Prompt: {prompt_template.format(subject)}")
    print(f"  Original: {target_true}")
    print(f"  New: {target_new}")
    
print("\n" + "=" * 80)


Selected 100 facts for editing:


Fact 1:
  Subject: Danielle Darrieux
  Prompt: The mother tongue of Danielle Darrieux is
  Original: French
  New: English

Fact 2:
  Subject: Edwin of Northumbria
  Prompt: The official religion of Edwin of Northumbria is
  Original: Christianity
  New: Islam

Fact 3:
  Subject: Toko Yasuda
  Prompt: Toko Yasuda, the
  Original: guitar
  New: piano

Fact 4:
  Subject: Autonomous University of Madrid
  Prompt: Autonomous University of Madrid, which is located in
  Original: Spain
  New: Sweden

Fact 5:
  Subject: Lyon
  Prompt: What is the twin city of Lyon? It is
  Original: Beirut
  New: Manila

Fact 6:
  Subject: Thomas Joannes Stieltjes
  Prompt: The mother tongue of Thomas Joannes Stieltjes is
  Original: Dutch
  New: English

Fact 7:
  Subject: Anaal Nathrakh
  Prompt: Anaal Nathrakh, that was created in
  Original: Birmingham
  New: Philadelphia

Fact 8:
  Subject: Apple A5
  Prompt: Apple A5 was created by
  Original: Apple
  New: Google

Fact 

## 4.2 Apply knowledge editing
Applying knowledge editing to facts.

In [ ]:
def apply_simple_edit(model, tokenizer, subject, prompt_template, target_new, target_true):
    """
    Apply a simple knowledge edit by updating model weights.
    This is a simplified version - for full ROME implementation, use the rome library.
    """
    # Format the prompt
    prompt = prompt_template.format(subject)
    
    # Tokenize
    inputs = tokenizer(prompt, return_tensors="pt")
    target_new_ids = tokenizer(target_new, return_tensors="pt", add_special_tokens=False)["input_ids"][0]
    target_true_ids = tokenizer(target_true, return_tensors="pt", add_special_tokens=False)["input_ids"][0]
    
    # Get the last token position
    prompt_length = inputs['input_ids'].shape[1]
    
    # Forward pass to get logits
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits[0, -1, :]  # Last token logits
        
        # Compute log probabilities
        log_probs = F.log_softmax(logits, dim=-1)
        
        # Get probabilities for target tokens
        target_new_prob = log_probs[target_new_ids].sum().item()
        target_true_prob = log_probs[target_true_ids].sum().item()
    
    # For a simple edit, we'll just track the change in probabilities
    # In full ROME, this would modify the model weights
    edit_info = {
        'prompt': prompt,
        'target_new': target_new,
        'target_true': target_true,
        'target_new_prob': target_new_prob,
        'target_true_prob': target_true_prob,
        'prob_change': target_new_prob - target_true_prob
    }
    
    return edit_info

# Apply edits to all xx facts
print("Applying knowledge edits to " + str(num_facts) +"facts...")
print("=" * 80)

edit_results = []

for idx, row in selected_facts.iterrows():
    rewrite = row['requested_rewrite']
    subject = rewrite['subject']
    prompt_template = rewrite['prompt']
    target_true = rewrite['target_true']['str']
    target_new = rewrite['target_new']['str']
    
    print(f"\nProcessing Fact {idx + 1}: {subject}")
    
    # Apply edit
    edit_info = apply_simple_edit(
        model, tokenizer, 
        subject, prompt_template, 
        target_new, target_true
    )
    
    edit_info['fact_id'] = idx + 1
    edit_info['subject'] = subject
    edit_results.append(edit_info)
    
    print(f"  Prompt: {edit_info['prompt']}")
    print(f"  Original ({target_true}): {edit_info['target_true_prob']:.4f}")
    print(f"  New ({target_new}): {edit_info['target_new_prob']:.4f}")
    print(f"  Change: {edit_info['prob_change']:.4f}")

print("\n" + "=" * 80)
print(f"\nCompleted editing {len(edit_results)} facts.")


Applying knowledge edits to 10 facts...

Processing Fact 1: Danielle Darrieux
  Prompt: The mother tongue of Danielle Darrieux is
  Original (French): -10.5633
  New (English): -11.9108
  Change: -1.3474

Processing Fact 2: Edwin of Northumbria
  Prompt: The official religion of Edwin of Northumbria is
  Original (Christianity): -31.3461
  New (Islam): -12.2204
  Change: 19.1256

Processing Fact 3: Toko Yasuda
  Prompt: Toko Yasuda, the
  Original (guitar): -40.7259
  New (piano): -21.8875
  Change: 18.8384

Processing Fact 4: Autonomous University of Madrid
  Prompt: Autonomous University of Madrid, which is located in
  Original (Spain): -12.4052
  New (Sweden): -35.9272
  Change: -23.5220

Processing Fact 5: Lyon
  Prompt: What is the twin city of Lyon? It is
  Original (Beirut): -47.6796
  New (Manila): -34.7592
  Change: 12.9204

Processing Fact 6: Thomas Joannes Stieltjes
  Prompt: The mother tongue of Thomas Joannes Stieltjes is
  Original (Dutch): -11.7199
  New (English): -11.

## 5. Evaluate the edited facts

In [14]:
# Evaluate all edits using the evaluation functions with dataset-specific prompts
print("Evaluating all edits...")
print("=" * 80)

def evaluate_with_dataset_prompts(model, tokenizer, row, target_new):
    """Evaluate using prompts from the CounterFact dataset."""
    rewrite = row['requested_rewrite']
    subject = rewrite['subject']
    prompt_template = rewrite['prompt']
    
    # Efficacy: Use generation prompts from dataset
    try:
        generation_prompts = row['generation_prompts']
    except (KeyError, IndexError):
        generation_prompts = []
    if len(generation_prompts) > 0:
        # Use first few generation prompts
        test_prompts = generation_prompts[:3] if len(generation_prompts) >= 3 else generation_prompts
    else:
        # Fallback to template
        test_prompts = [prompt_template.format(subject)]
    
    efficacy_scores = []
    for prompt in test_prompts:
        score = compute_log_probs(model, tokenizer, prompt, target_new)
        efficacy_scores.append(score)
    efficacy = np.mean(efficacy_scores) if efficacy_scores else 0.0
    
    # Paragraph: Use paraphrase prompts
    try:
        paraphrase_prompts = row['paraphrase_prompts']
    except (KeyError, IndexError):
        paraphrase_prompts = []
    if len(paraphrase_prompts) > 0:
        # Use first paraphrase prompt for paragraph generation
        para_prompt = paraphrase_prompts[0]
    else:
        para_prompt = f"The {prompt_template.format(subject)} is {target_new}."
    
    inputs = tokenizer(para_prompt, return_tensors="pt")
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_length=100,
            num_return_sequences=1,
            temperature=0.7,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id if tokenizer.eos_token_id is not None else tokenizer.pad_token_id
        )
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    paragraph = 1.0 if target_new.lower() in generated_text.lower() else 0.0
    
    # Neighborhood: Use neighborhood prompts from dataset
    # These prompts test similar but unrelated facts that should NOT be affected by the edit
    try:
        neighborhood_prompts = row['neighborhood_prompts']
    except (KeyError, IndexError):
        neighborhood_prompts = []
    if len(neighborhood_prompts) > 0:
        # For neighborhood, we want to ensure the model still works on similar prompts
        # We'll compute average log probability on these prompts (higher is better)
        # This is a simplified metric - in practice, you'd check specific expected outputs
        neighborhood_scores = []
        for n_prompt in neighborhood_prompts[:5]:  # Use first 5
            # Get the log probability of the most likely token (as a proxy for model confidence)
            inputs = tokenizer(n_prompt, return_tensors="pt")
            with torch.no_grad():
                outputs = model(**inputs)
                logits = outputs.logits[0, -1, :]
                log_probs = F.log_softmax(logits, dim=-1)
                # Use max log prob as a measure of model confidence
                max_log_prob = log_probs.max().item()
                neighborhood_scores.append(max_log_prob)
        neighborhood = np.mean(neighborhood_scores) if neighborhood_scores else 0.0
    else:
        # Fallback to default neighborhood score
        neighborhood = neighborhood_score(model, tokenizer, subject, "attribute", target_new)
    
    return efficacy, paragraph, neighborhood

evaluation_results = []

for idx, edit_info in enumerate(edit_results):
    fact_id = edit_info['fact_id']
    subject = edit_info['subject']
    target_new = edit_info['target_new']
    row = selected_facts.iloc[idx]
    
    print(f"\nEvaluating Fact {fact_id}: {subject}")
    
    # Evaluate using dataset-specific prompts
    efficacy, paragraph, neighborhood = evaluate_with_dataset_prompts(
        model, tokenizer, row, target_new
    )
    
    result = {
        'fact_id': fact_id,
        'subject': subject,
        'target_new': target_new,
        'efficacy': efficacy,
        'paragraph': paragraph,
        'neighborhood': neighborhood
    }
    evaluation_results.append(result)
    
    print(f"  Efficacy: {efficacy:.4f}")
    print(f"  Paragraph: {paragraph:.4f}")
    print(f"  Neighborhood: {neighborhood:.4f}")

print("\n" + "=" * 80)


Evaluating all edits...

Evaluating Fact 1: Danielle Darrieux
  Efficacy: -12.0365
  Paragraph: 0.0000
  Neighborhood: -1.5798

Evaluating Fact 2: Edwin of Northumbria
  Efficacy: -13.2924
  Paragraph: 0.0000
  Neighborhood: -1.0655

Evaluating Fact 3: Toko Yasuda
  Efficacy: -29.3544
  Paragraph: 0.0000
  Neighborhood: -2.5226

Evaluating Fact 4: Autonomous University of Madrid
  Efficacy: -31.2185
  Paragraph: 0.0000
  Neighborhood: -1.5122

Evaluating Fact 5: Lyon
  Efficacy: -32.5958
  Paragraph: 0.0000
  Neighborhood: -1.8562

Evaluating Fact 6: Thomas Joannes Stieltjes
  Efficacy: -12.2375
  Paragraph: 0.0000
  Neighborhood: -2.1002

Evaluating Fact 7: Anaal Nathrakh
  Efficacy: -15.2167
  Paragraph: 0.0000
  Neighborhood: -1.7725

Evaluating Fact 8: Apple A5
  Efficacy: -11.2779
  Paragraph: 0.0000
  Neighborhood: -2.4017

Evaluating Fact 9: Wellington
  Efficacy: -33.4749
  Paragraph: 0.0000
  Neighborhood: -1.8593

Evaluating Fact 10: Shree Pundalik
  Efficacy: -28.7709
  Para

In [15]:
# Summary of all edits
print("Summary of Knowledge Editing Results")
print("=" * 80)

results_df = pd.DataFrame(evaluation_results)
print("\nEvaluation Metrics Summary:")
print(results_df[['fact_id', 'subject', 'target_new', 'efficacy', 'paragraph', 'neighborhood']].to_string(index=False))

print("\n" + "=" * 80)
print("\nAverage Scores:")
print(f"  Average Efficacy: {results_df['efficacy'].mean():.4f}")
print(f"  Average Paragraph: {results_df['paragraph'].mean():.4f}")
print(f"  Average Neighborhood: {results_df['neighborhood'].mean():.4f}")

print("\n" + "=" * 80)
print(f"\nSuccessfully edited and evaluated {len(evaluation_results)} facts from CounterFact dataset.")


Summary of Knowledge Editing Results

Evaluation Metrics Summary:
 fact_id                         subject   target_new   efficacy  paragraph  neighborhood
       1               Danielle Darrieux      English -12.036507        0.0     -1.579840
       2            Edwin of Northumbria        Islam -13.292429        0.0     -1.065450
       3                     Toko Yasuda        piano -29.354420        0.0     -2.522639
       4 Autonomous University of Madrid       Sweden -31.218536        0.0     -1.512250
       5                            Lyon       Manila -32.595797        0.0     -1.856241
       6        Thomas Joannes Stieltjes      English -12.237504        0.0     -2.100247
       7                  Anaal Nathrakh Philadelphia -15.216739        0.0     -1.772492
       8                        Apple A5       Google -11.277897        0.0     -2.401666
       9                      Wellington    Sheffield -33.474876        0.0     -1.859256
      10                  Shree Pu

## 6. Apply ROME
Use the ROME method to apply the edit. This requires the ROME implementation.


In [7]:
# Placeholder for ROME application
# from rome import apply_rome_edit

# edit_request = {
#     "subject": subject,
#     "relation": relation,
#     "new_object": new_object
# }

# model = apply_rome_edit(model, edit_request)

print("ROME edit applied (placeholder)")


ROME edit applied (placeholder)


## 7. Validate the Edit
Compare model outputs before and after the edit.


In [9]:
def generate_text(prompt):
    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(**inputs, max_length=50)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Test before/after edit
prompt = f"What is the capital of {subject}?"
print("Prompt:", prompt)
print("Model response:", generate_text(prompt))


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Prompt: What is the capital of Shree Pundalik?
Model response: What is the capital of Shree Pundalik?

Shree Pundalik is a village in the Shree Pundalik district in the Indian state of West Bengal. It is located in the Shree Pundal
